In [ ]:
%%bigquery

SELECT
  p1.full_name AS person1_name,
  p2.full_name AS person2_name,
  t1.themes,
  t1.themes_summary
FROM
  `graph-demo-471710`.`gdelt`.`person_cooccurrence` AS t1
JOIN
  `graph-demo-471710`.`gdelt`.`person` AS p1
ON
  t1.person1_id = p1.person_id
JOIN
  `graph-demo-471710`.`gdelt`.`person` AS p2
ON
  t1.person2_id = p2.person_id
LIMIT
  1000;

Query is running:   0%|          |

Downloading:   0%|          |

,person1_name,person2_name,themes,themes_summary
0,Trent Nelson,Melania Trump,"[TAX_FNCACT_LADY, EDUCATION, SOC_POINTSOFINTER...",None
1,Trent Nelson,Nigel Farage,"[SOC_POINTSOFINTEREST_UNIVERSITY, EDUCATION, W...",None
2,Sam Miltich,Gary Schulte,"[TAX_WORLDLANGUAGES_ROMANI, TAX_ETHNICITY_ROMA...",None
3,Sam Miltich,Matthew Miltich,"[TAX_WORLDLANGUAGES_ROMANI, TAX_ETHNICITY_ROMA...",None
4,Sam Miltich,Mark Kreitzer,"[TAX_WORLDLANGUAGES_ROMANI, TAX_ETHNICITY_ROMA...",None
...,...,...,...,...
995,Eng Abdulmutallab,Ahmad Abu,"[TAX_FNCACT_CITIZENS, TAX_ETHNICITY_MUSLIMS, T...",None
996,Eng Abdulmutallab,Eng Muhammad Ibrahim,"[AGRICULTURE, TAX_FNCACT_MANAGER, TAX_FNCACT_P...",None
997,Eng Abdulmutallab,Abdul Nasser Sayeh,"[AGRICULTURE, TAX_FNCACT_POLICE, CRISISLEX_C07...",None
998,Eng Abdulmutallab,Ahmad Hawari,"[AGRICULTURE, SECURITY_SERVICES, CRISISLEX_C07...",None


# Create Tables: create tables for nodes and edges.

In [ ]:
%%bigquery --project graph-demo-471710
DROP PROPERTY GRAPH IF EXISTS gdelt.GdeltGraph;

DROP TABLE IF EXISTS `graph-demo-471710.gdelt.person`;
DROP TABLE IF EXISTS `graph-demo-471710.gdelt.person_cooccurrence`;

CREATE TABLE IF NOT EXISTS graph-demo-471710.gdelt.person (
    person_id STRING NOT NULL OPTIONS(description="Logical Primary Key. Unique identifier for the person."),
    name STRING NOT NULL OPTIONS(description="The common name of the person."),
    first_name STRING,
    last_name STRING,
    full_name STRING,
    name_variations ARRAY<STRING> OPTIONS(description="Known variations or aliases of the person's name."),
    first_seen_date DATE OPTIONS(description="The date the person was first mentioned."),
    last_seen_date DATE OPTIONS(description="The date the person was last mentioned."),
    total_mentions INT64 OPTIONS(description="A total count of the person's mentions."),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP() OPTIONS(description="Timestamp when the record was created."),
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP() OPTIONS(description="Timestamp when the record was last updated."),
    PRIMARY KEY (person_id) NOT ENFORCED
)
CLUSTER BY person_id, name
OPTIONS(
    description="Node table containing master list of all identified individuals."
);

CREATE TABLE IF NOT EXISTS `graph-demo-471710.gdelt.person_cooccurrence` (
    relationship_id STRING NOT NULL OPTIONS(description="Logical Primary Key. Unique identifier for this specific co-occurrence."),
    person1_id STRING NOT NULL OPTIONS(description="Logical Foreign Key referencing person_id in the person table."),
    person2_id STRING NOT NULL OPTIONS(description="Logical Foreign Key referencing person_id in the person table."),
    cooccurrence_count INT64 OPTIONS(description="The number of times these two people were mentioned together."),
    first_cooccurrence_date DATE OPTIONS(description="The date of the first joint mention."),
    last_cooccurrence_date DATE OPTIONS(description="The date of the most recent joint mention."),
    article_ids ARRAY<STRING> OPTIONS(description="A list of article IDs where the co-occurrence was found."),
    themes ARRAY<STRING> OPTIONS(description="A list of themes associated with their joint mentions."),
    themes_summary STRING OPTIONS(description="A summary of the common themes in their co-occurrence."),
    strength_score FLOAT64 OPTIONS(description="A calculated score representing the strength of the relationship."),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP() OPTIONS(description="Timestamp when the record was created."),
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP() OPTIONS(description="Timestamp when the record was last updated."),
    PRIMARY KEY (relationship_id) NOT ENFORCED,
    CONSTRAINT fk_person1 FOREIGN KEY (person1_id) REFERENCES `graph-demo-471710.gdelt.person` (person_id) NOT ENFORCED,
    CONSTRAINT fk_person2 FOREIGN KEY (person2_id) REFERENCES `graph-demo-471710.gdelt.person` (person_id) NOT ENFORCED
)
CLUSTER BY person1_id, person2_id
OPTIONS(
    description="Edge table storing the relationships (co-occurrences) between individuals from the person table."
);

Query is running:   0%|          |

""


# Insert Data

In [ ]:
%%bigquery

INSERT INTO `graph-demo-471710.gdelt.person`
        (person_id, name, first_name, last_name, full_name, first_seen_date, last_seen_date, total_mentions)
        WITH CleanedPersons AS (
          SELECT DISTINCT
            REGEXP_REPLACE(REGEXP_REPLACE(person, r',.*', ''), r'^A ', '') as clean_name,
            CASE
              WHEN ARRAY_LENGTH(SPLIT(REGEXP_REPLACE(REGEXP_REPLACE(person, r',.*', ''), r'^A ', ''), ' ')) > 0
              THEN SPLIT(REGEXP_REPLACE(REGEXP_REPLACE(person, r',.*', ''), r'^A ', ''), ' ')[OFFSET(0)]
              ELSE NULL
            END as first_name,
            CASE
              WHEN ARRAY_LENGTH(SPLIT(REGEXP_REPLACE(REGEXP_REPLACE(person, r',.*', ''), r'^A ', ''), ' ')) > 1
              THEN SPLIT(REGEXP_REPLACE(REGEXP_REPLACE(person, r',.*', ''), r'^A ', ''), ' ')[OFFSET(ARRAY_LENGTH(SPLIT(REGEXP_REPLACE(REGEXP_REPLACE(person, r',.*', ''), r'^A ', ''), ' ')) - 1)]
              ELSE NULL
            END as last_name,
            REGEXP_REPLACE(REGEXP_REPLACE(person, r',.*', ''), r'^A ', '') as full_name
          FROM `graph-demo-471710.gdelt..gkg_partitioned`,
          UNNEST(SPLIT(V2Persons, ';')) AS person
          WHERE V2Persons IS NOT NULL AND V2Persons != '' AND REGEXP_REPLACE(REGEXP_REPLACE(person, r',.*', ''), r'^A ', '') != ''
        ),
        PersonCounts AS (
          SELECT
            REGEXP_REPLACE(REGEXP_REPLACE(person, r',.*', ''), r'^A ', '') as clean_name,
            COUNT(*) as total_mentions
          FROM `graph-demo-471710.gdelt..gkg_partitioned`,
          UNNEST(SPLIT(V2Persons, ';')) AS person
          WHERE V2Persons IS NOT NULL AND V2Persons != '' AND REGEXP_REPLACE(REGEXP_REPLACE(person, r',.*', ''), r'^A ', '') != ''
          GROUP BY REGEXP_REPLACE(REGEXP_REPLACE(person, r',.*', ''), r'^A ', '')
        )
        SELECT
          GENERATE_UUID() as person_id,
          cp.clean_name as name,
          cp.first_name,
          cp.last_name,
          cp.full_name,
          CURRENT_DATE() as first_seen_date,
          CURRENT_DATE() as last_seen_date,
          pc.total_mentions
        FROM CleanedPersons cp
        JOIN PersonCounts pc ON cp.clean_name = pc.clean_name;

INSERT INTO graph-demo-471710.gdelt.person_cooccurrence
(relationship_id, person1_id, person2_id, cooccurrence_count, first_cooccurrence_date, last_cooccurrence_date, article_ids, themes)

WITH PersonPairs AS (
  /* 1. PARSING (Canonical Only)
     We keep 'a < b' here to do the heavy string parsing only ONCE per pair.
  */
  SELECT DISTINCT
    g.GKGRECORDID,
    g.V2Themes,
    REGEXP_REPLACE(REGEXP_REPLACE(a, r',.*', ''), r'^A ', '') AS name1,
    REGEXP_REPLACE(REGEXP_REPLACE(b, r',.*', ''), r'^A ', '') AS name2,
    PARSE_DATE('%Y%m%d', SUBSTR(CAST(DATE AS STRING), 1, 8)) as cooccurrence_date
  FROM graph-demo-471710.gdelt.gkg_partitioned g,
  UNNEST(SPLIT(V2Persons, ';')) AS a,
  UNNEST(SPLIT(V2Persons, ';')) AS b
  WHERE a < b  -- Keep canonical order to avoid duplicate processing here
    AND REGEXP_REPLACE(REGEXP_REPLACE(a, r',.*', ''), r'^A ', '') != ''
    AND REGEXP_REPLACE(REGEXP_REPLACE(b, r',.*', ''), r'^A ', '') != ''
),

PersonCooccurrence AS (
  /* 2. COUNTING (Bidirectional Expansion)
     We aggregate the stats, then use UNION ALL to create the reverse edges.
  */
  -- Forward Edges (A -> B)
  SELECT
    p1.person_id as person1_id,
    p2.person_id as person2_id,
    COUNT(*) as cooccurrence_count,
    MIN(cooccurrence_date) as first_cooccurrence_date,
    MAX(cooccurrence_date) as last_cooccurrence_date,
    ARRAY_AGG(DISTINCT GKGRECORDID) as article_ids
  FROM PersonPairs pp
  JOIN graph-demo-471710.gdelt.person p1 ON pp.name1 = p1.name
  JOIN graph-demo-471710.gdelt.person p2 ON pp.name2 = p2.name
  GROUP BY p1.person_id, p2.person_id

  UNION ALL

  -- Reverse Edges (B -> A)
  SELECT
    p2.person_id as person1_id, -- Swap: p2 is now Source
    p1.person_id as person2_id, -- Swap: p1 is now Target
    COUNT(*) as cooccurrence_count,
    MIN(cooccurrence_date) as first_cooccurrence_date,
    MAX(cooccurrence_date) as last_cooccurrence_date,
    ARRAY_AGG(DISTINCT GKGRECORDID) as article_ids
  FROM PersonPairs pp
  JOIN graph-demo-471710.gdelt.person p1 ON pp.name1 = p1.name
  JOIN graph-demo-471710.gdelt.person p2 ON pp.name2 = p2.name
  GROUP BY p1.person_id, p2.person_id
),

PersonThemesCanonical AS (
  /* 3. THEMES (Calculate Once)
     Calculating themes is expensive (string parsing). We do it once for the canonical pairs.
  */
  SELECT
    p1.person_id as person1_id,
    p2.person_id as person2_id,
    ARRAY_AGG(
      theme
      ORDER BY theme_count DESC
      LIMIT 20
    ) as themes
  FROM (
    SELECT
      pp.name1,
      pp.name2,
      REGEXP_REPLACE(theme, r',.*', '') as theme,
      COUNT(*) as theme_count
    FROM PersonPairs pp,
    UNNEST(SPLIT(V2Themes, ';')) AS theme
    WHERE theme IS NOT NULL AND theme != ''
    GROUP BY pp.name1, pp.name2, REGEXP_REPLACE(theme, r',.*', '')
  ) theme_counts
  JOIN graph-demo-471710.gdelt.person p1 ON theme_counts.name1 = p1.name
  JOIN graph-demo-471710.gdelt.person p2 ON theme_counts.name2 = p2.name
  GROUP BY p1.person_id, p2.person_id
),

PersonThemesBidirectional AS (
  /* 4. THEMES (Bidirectional Expansion)
     We flip the canonical themes so the LEFT JOIN below works for both directions.
  */
  SELECT person1_id, person2_id, themes FROM PersonThemesCanonical
  UNION ALL
  SELECT person2_id as person1_id, person1_id as person2_id, themes FROM PersonThemesCanonical
)

/* 5. FINAL JOIN
   Now we have A->B and B->A for both stats and themes. The join is simple and fast.
*/
SELECT
  GENERATE_UUID() as relationship_id,
  pc.person1_id,
  pc.person2_id,
  pc.cooccurrence_count,
  pc.first_cooccurrence_date,
  pc.last_cooccurrence_date,
  pc.article_ids,
  COALESCE(pt.themes, []) as themes
FROM PersonCooccurrence pc
LEFT JOIN PersonThemesBidirectional pt
  ON pc.person1_id = pt.person1_id
  AND pc.person2_id = pt.person2_id
WHERE pc.person1_id != pc.person2_id;


Query is running:   0%|          |

""


# Create Property Graph

In [ ]:
%%bigquery

CREATE PROPERTY GRAPH gdelt.GdeltGraph
 NODE TABLES (
   gdelt.person
     KEY (person_id)
     LABEL Person
     PROPERTIES ( person_id,
                  name,
                  first_name,
                  last_name,
                  full_name,
                  name_variations,
                  first_seen_date,
                  last_seen_date,
                  total_mentions,
                  created_at,
                  updated_at)
 )

 EDGE TABLES(
   gdelt.person_cooccurrence
     KEY (relationship_id)
     SOURCE KEY (person1_id) REFERENCES Person (person_id)
     DESTINATION KEY (person2_id) REFERENCES Person (person_id)
     LABEL CO_OCCURS_WITH
     PROPERTIES ( cooccurrence_count,
                  first_cooccurrence_date,
                  last_cooccurrence_date,
                  article_ids,
                  strength_score,
                  themes,
                  themes_summary,
                  created_at,
                  updated_at)
 );

Query is running:   0%|          |

""


# Query Graph

In [ ]:
%%bigquery
MATCH
 (person:Person {name: "Jacoby"})-[own:Owns]->
 (account:Account)-[repay:Repays]->(loan:Loan)
RETURN
 account.id AS account_id,
 own.id AS own_id,
 repay.create_time AS repay_time,
 repay.amount AS loan_repay_amount,
 loan.id AS loan_id,
 loan.loan_amount AS loan_amount
ORDER BY repay.create_time;

Executing query with job ID: f393cbec-e31c-42ae-ac3d-b0ddbcdb4f2b
Query executing: 0.12s


ERROR:
 400 Syntax error: Unexpected keyword MATCH at [1:1]; reason: invalidQuery, location: query, message: Syntax error: Unexpected keyword MATCH at [1:1]

Location: US
Job ID: f393cbec-e31c-42ae-ac3d-b0ddbcdb4f2b



In [ ]:
%%bigquery --graph

--DECLARE search_name STRING DEFAULT 'David Bowie';
GRAPH gdelt.GdeltGraph
MATCH
 (p1:Person)-[co1:CO_OCCURS_WITH]->(p2:Person)
WHERE (p1.full_name = 'David Bowie')
AND co1.cooccurrence_count > 10
RETURN p1.full_name as person1, p2.full_name as person2, co1.cooccurrence_count as co_occurrence_count
ORDER BY co1.cooccurrence_count DESC
LIMIT 500;

Query is running:   0%|          |

Downloading:   0%|          |

,person1,person2,co_occurrence_count
0,David Bowie,Madeleine Haddon,56
1,David Bowie,Alexander Mcqueen,46
2,David Bowie,David Jones,46
3,David Bowie,Nile Rodgers,45
4,David Bowie,Iggy Pop,45
5,David Bowie,Kansai Yamamoto,27
6,David Bowie,Harriet Reed,25
7,David Bowie,Miles Davis,23
8,David Bowie,Haywood Stenton Jones,23
9,David Bowie,Sabrina Offord,23
